Project Report for MAP543 - Database Management II  
Authors: Roberta Conrad, Dora Ranilovic, Zigfrid Zvezdin  
dependecies: Python 3.6.4+

# TF IDF Report

## Table of Content 

1. Map Reduce Algorithm: a description of the _adopted solution_

    1a. Other designed algorithms plus related global comments/description including comments to the code 

3. Experimental analysis, concerning in particular scalability 

    2a. Comments about the experimental analysis outlining weak and strong points of the algorithms.

4. an appendix including all the code

# 1. MapReduce algorithm
calculate TF-IDF scores given an input set of documents. Provide both a MapReduce Python Hadoop streaming and a Spark implementation. 
## 1.a. Adopted Version

### Python Hadoop

The first implementation of Tf-Idf is a tradidional MapReduce algorithm, designed to be run on a Hadoop cluster with only Python 2.7+ dependencies. It consists of consecutive Map and Reduce jobs, take a corpus of text documents as the input and compute the (nonzero) Tf-Idf scores for each word-document pair in the corpus. The following sections describe the code of all three jobs of the algorithm in detail, as well as the inputs and outputs. As this approach utilizes the traditional MapReduce framework, the inputs and outputs are always key-value pairs written in the format <(key, value)>. Aditionally, the output of each *reduce* step, is the raw input of the *map* step of the following job. 

#### Job 1: Map

__Input__: (doc, contents)

__Output__: ((doc, term), 1)

__Description__: Read text document line by line, envoking the source document name. Split each line into terms (words), convert to lowercase and remove punctuation, and emit the document name and term, together with the digit 1. This choice of output was made so that after the shuffle and sort step, the documents and identical words in each document will be grouped together to be able to perform two types of counts.

In [ ]:
import os
import sys

for line in sys.stdin:
    line = line.strip() #remove whitespace
    terms = line.split(" ") #create list of words
    path = os.environ['mapreduce_map_input_file'].split("/") #get full path of the text document
    docname = path[-1]
    for term in terms:
        term = term.strip('''!()-[]{};:'",<>./?@#$%^&*_~''').lower()
        print('%s\t%s' % (docname + '_' + term, 1))

#### Job 1: Reduce

__Input__: ((doc, term), 1)

__Output__: ((term, doc), (N, n))

__Description__: This reduce step is computationally the most intensive as it performs two important types of counts at the same time: unique term per document (N) and term count (n). These two numbers together form the term freqency (Tf) part of the Tf-Idf expression. They are computed using nested loops and local lists, and taking advantage of the sorted nature of the inputs.
The term count is accosiated to each (term, document) pair and is computed first, in the inside loop: the memory of the one previous term is kept and compared to the current term; if they match the counter is incremented an we move to the next term; once they no longer match, the term and its count are saved in a list, holding all terms in the current document. Once the end of the document is reached, all terms in the list are emitted one-by-one, along with the document name, their respective counts in the document and the length of the final list, which represents the number of unique terms in the document.

In [ ]:
import sys

current_doc = None #Initialize helper variables
current_term = None
term_list = []
current_term_count = 0

for line in sys.stdin:
    line = line.strip()
    pair, count = line.split('\t', 1)
    doc, term = pair.split('_', 1)

    try: # convert count (currently a string) to int
        count = int(count)
    except ValueError:
        continue  # count was not a number, so silently ignore/discard this line

    if current_doc == doc: #outside loop, check if still in the same document
        if current_term == term:
            current_term_count += count
        else:
            term_list.append((current_term, current_term_count)) #hold term anf final count
            current_term = term
            current_term_count = count

    else:
        term_list.append((current_term, current_term_count)) #end of doc; emit all terms and counts
        if current_doc:
            for t, ct in term_list:
                print('%s\t%s' % (t+'_'+current_doc,
                                  str(len(term_list))+'_'+(str(ct))))

        current_term_count = count #reset counters
        current_doc = doc
        current_term = term
        term_list = [] #reset list

if term_list: #emit all terms except last in last doc
    for t, ct in term_list:
        print('%s\t%s' % (t+'_'+current_doc,
                          str(len(term_list)+1)+'_'+(str(ct))))

print('%s\t%s' % (current_term+'_'+current_doc, #emit last term in last doc
                  str(len(term_list)+1)+'_'+str(current_term_count)))

#### Job 2: Map

__Input__: ((term, doc), (N, n))

__Output__: (term, (docname, N, n, 1))

__Description__: The main purpose of this *map* step is to uncouple the key and pass only the term as the key to the next reducer, as we now want to calculate the document frequency of each term, and therefore need the terms grouped. Although we would achieve the same grouping by keeping the document name in the key (separated by an underscore, for example, from the term), we chose to perform the separation step in the mapper in order to take advantage of parallelization, and avoid performing the same task in the reducer.

In [ ]:
import sys
for line in sys.stdin:
    pair, vals = line.split('\t')
    term, docname = pair.split('_', 1)
    N, n = vals.split('_')
    try:
        n = int(n)
        N = int(N)
    except ValueError:
        continue
    print('%s\t%s' % (term, docname+'_'+str(N)+'_'+str(n)+'_'+str(1)))

#### Job 2: Reduce

__Input__: (term, (docname, N, n, 1))

__Output__: ((term,docname), (N,n,d))

__Description__: Much like a simple word count, count the occurence of each term, which thanks to the previous reducer will occur once per each document it appears in. Since we need the rest of the information found in the values of the key, we need to keep them in a local list until we reach the last occurence of the term, at which point we emit all term-document pairs for the current term.

In [ ]:
import sys
current_term = None
doc_list = []
current_count = 0
term = None
doc = None

for line in sys.stdin:
    term, rest = line.split('\t', 1)
    doc, N, n, count = rest.rsplit('_', 3)
    
    try: # convert count (currently a string) to int
        count = int(count)
    except ValueError:
        # count was not a number, so silently ignore/discard this line
        continue
        
    if current_term == term:
        doc_list.append((doc, N, n))
        current_count += count
    else:
        if current_term:
            for document, N, n in doc_list:
                print(
                    '%s\t%s' % (current_term + '_' + document,
                                str(N)+'_' + str(n) + '_' + str(current_count)))
            doc_list = []
            n_list = []
        current_count = count
        current_term = term
        doc_list.append((doc, N, n))
if current_term == term: #emit last term
    for document, N, n in doc_list:
        print('%s\t%s' % (current_term + '_' + document,
                          str(N)+'_' + str(n) + '_' + str(current_count)))


#### Job 3: Map

__Input__: ((term, docname), (N, n, d))

__Output__: ((term, docname), tfidf)

__Description__: Having all necessary elements, compute the final Tf-Idf score for each (term, document) pair.

In [2]:
import sys
import math

for line in sys.stdin:

    pair, vals = line.split('\t', 1)
    N, n, d = vals.split('_', 2)
    try:
        N = int(N)
        n = int(n)
        d = int(d)
    except ValueError:
        continue
    tf = n/N
    idf = math.log(10000/(1+d)) # use 10,000 as max number of documents
    tfidf = tf * idf
    print('%s\t%s' % (pair, tfidf))

#### Job 3: Reduce

__Input__: ((term, docname), tfidf)

__Output__: ((term, docname), tfidf)

__Description__: Analogous to a pass task, simply emit the input without any additional tratment.  

In [3]:
import sys

for line in sys.stdin:
    print(line)

### Spark

## 1.b. Other designed algorithms

### Spark


# 2. Experimental analysis

In order to compare the performance of the three different implementations, we designed two different tests considering different structures of a text collection with increasing sizes. We differentiated between a collection of documents containing many short documents; and a collection containing few long documents:

+ Collections with a fixed document size: we fixed the number of words at 100 words per document, changing the number of documents in (100,200,300,400,500,1000).  
+ Collections with a fixed number of documents: We ran our tf-idf algorithms on 100 documents containing (100,200,300,400,500,1000) words. 

For easy reproducability this report contains all code and scripts that have been used to produce the following analysis and can be recreated on AWS Elastic Map Reduce cluster. 

*Limitations*  
In order to control the size of the documents used to test our algorithm, we decided to write a script creating documents for testing purposes. One document containing 100 words has a size of 11 KB when created by our script. 

## 2.1. Comparison of the implementations

The second Spark implementation outperforms the other algorithms for almost all input sizes and scales as document size and/or number increase. We prefer the second Spark algorithm for both a few, long documents, as well as many very short ones - such as tweets or emails, for example. 

The first Spark algorithm is the most efficient algorithm for a very small text collection. However, it does not scale with size and turns out to be very slow for increasingly larger document sizes, while it shows the exact same performance as the fist Spark implementation for many small documents. 

The MapReduce implementation is the slowest option of all three and only better then the first Spark implementation as documents become longer then 300 Words (approx. 30 KB). 

<img src="./longdoc.PNG">

<img src="./Manydoc.PNG">


### 2.2. Practical Instructions to run tests on the cluster

1. Create text documents on hadoop home directory  
    + create the input directory on /user/hadoop local file system 
    + in /input, import the python script and execute the text_generator function to create documents needed, specify params as described below. 
    + move only the .txt to the hdfs hadoop input directory 

2. Download and run the MapReduce algorithms
 + give permission to access all documents
 + execute all jobs through bash script runjobs.sh
 + after retrieving the time information, clear input directories with clear.sh

# 3. Code Appendix

### text_generator.py
*Requirements* Requires installing the nlkt library brown.   

*Functionality* The function create_docs takes as input parameters the number of words per document, number of documents, and start number of the document to be taken into account when naming files. The function creates the specified number of documents at the specified length by taking samples of the brown corpus. It randomly samples lines of twenty words at the same time.

The Brown University Standard Corpus of Present-Day American English (or just Brown Corpus) was compiled in the 1960s as a general corpus. It contains 500 samples of English-language text, totaling roughly one million words, compiled from works published in the United States in 1961. It is a suitable library to sample from in order to test our algorithm as it represents how language is used in reality. 

In [ ]:
#!/usr/bin/python
from nltk.corpus import brown
import random
corpus_length = len(brown.words())
hardcopy = brown.words()


def create_docs(number_of_words_per_doc=200, num_doc=10, startnr=0):
    # control number of words per doc
    # and number of documents
    # we fix the line length at 20
    line_length = 20
    number_of_lines = int(number_of_words_per_doc / line_length)

    for i in range(0, num_doc):
        # create new file with writing + permission
        new_file = open("textdoc"+str(number_of_words_per_doc) +
                        "words" + str(i+startnr)+".txt", "w+")
        for line in range(0, number_of_lines):
            words = list(map(
                lambda x: hardcopy[x:x+line_length], random.sample(range(corpus_length), line_length)))
            sentences = list(
                map(lambda x: ' '.join(word for word in x), words))
            text = ''.join(map(str, sentences))
            new_file.write(text + "\n")
        new_file.close()
        if (i % 10 == 0):
            print("You created "+str(i)+" files! "+str(num_doc-i)+" left")